In [1]:
pip install folium

In [2]:
pip install pyproj

In [146]:
import pandas as pd
import numpy as np
import pyproj
import folium

local_point=pd.read_csv('서울시 민방위대피소.csv',encoding='cp949')
local_gangseo=pd.read_csv('서울특별시 강서구 민방위대피시설 인허가 정보.csv',encoding='cp949')
local_seocho=pd.read_csv('서울특별시 서초구 민방위대피시설 인허가 정보.csv',encoding='cp949')

## @서초 강서 파일 전처리 시작@

### 1. 강서, 서초 파일 합침

In [147]:
local_gangcho = pd.concat([local_gangseo,local_seocho])

###  2.사용중인 것만 가져오기

In [148]:
local_gangcho_1 = local_gangcho[local_gangcho['상세영업상태명'] =='사용중']

In [149]:
local_gangcho_2 = local_gangcho_1.reset_index(drop=True)
local_gangcho_2

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),비상시설위치,시설구분명,시설명_건물명,해제일자
0,3150000,3150000-S100900001,19990901,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20210426152523,U,2021-04-28 02:40:00.0,NaN,187321.993372,447803.960023,서울특별시 강서구 화곡동 836번지 13호,공공시설,성석교회 지하주차장,NaN
1,3150000,3150000-S199000004,19900901,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20220112153153,U,2022-01-14 02:40:00.0,NaN,186557.645724,447487.800455,서울특별시 강서구 화곡동 893번지 1호,공공시설,KT 화곡지사 지하주차장,NaN
2,3150000,3150000-S199600003,19961206,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20220118181506,U,2022-01-20 02:40:00.0,NaN,185617.497942,449153.637972,서울특별시 강서구 화곡동 1026번지 12호,공공시설,서울큰나무병원,NaN
3,3150000,3150000-S199900275,19990901,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20191204135513,U,2019-12-06 02:40:00.0,NaN,186645.815384,449856.896042,서울특별시 강서구 화곡동 980번지 16호,공공시설,강서구청 지하상황실,NaN
4,3150000,3150000-S199900277,19990901,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20200715165443,U,2020-07-17 02:40:00.0,NaN,188295.158517,449716.240275,서울특별시 강서구 염창동 275번지 12호,공공시설,강서보건소,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,3210000,3210000-S201900002,20191202,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20211014173609,U,2021-10-16 02:40:00.0,NaN,205634.311120,438769.485853,서울특별시 서초구 내곡동 77번지,공공시설,서초더샵포레아파트,NaN
320,3210000,3210000-S201900003,20191202,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20211014173623,U,2021-10-16 02:40:00.0,NaN,205525.422778,439208.962103,서울특별시 서초구 내곡동 143번지,공공시설,서초포레스타 2단지아파트,NaN
321,3210000,3210000-S201900004,20191202,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20211014173635,U,2021-10-16 02:40:00.0,NaN,204921.170289,438918.090853,서울특별시 서초구 신원동 374번지,공공시설,서초포레스타 5단지아파트,NaN
322,3210000,3210000-S201900005,20191202,NaN,1,영업/정상,18,사용중,NaN,NaN,...,20211014173647,U,2021-10-16 02:40:00.0,NaN,204819.172335,438685.126669,서울특별시 서초구 신원동 285번지,공공시설,서초포레스타 6단지아파트,NaN


### 3. 전체 민방위 데이터와 같은 컬럼명으로 생성

In [150]:
local_gangcho_2['대피시설명'] = local_gangcho_2['시설명_건물명']
local_gangcho_2['주소'] = local_gangcho_2['비상시설위치']
local_gangcho_2['평시활용 유형'] = local_gangcho_2['해제일자']
local_gangcho_2['대피시설 면적'] = local_gangcho_2['소재지면적']

### 4. 좌표계, 경위도로 변환

In [151]:
del_list = set()
for i in local_gangcho_2['좌표정보(X)'][local_gangcho_2['좌표정보(X)'].isnull()].index :
    del_list.add(i)
for i in local_gangcho_2['좌표정보(Y)'][local_gangcho_2['좌표정보(Y)'].isnull()].index :
    del_list.add(i)

In [152]:
del_list

{166, 167, 168, 169, 170}

In [153]:
local_gangcho_3 = local_gangcho_2.drop(del_list,axis=0)

In [154]:
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

In [155]:
local_gangcho_3_1=local_gangcho_3.loc[:,'좌표정보(X)':'좌표정보(Y)']
coord=np.array(local_gangcho_3_1)
p1_type = "epsg:2097"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

C:\Users\i\Anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\i\Anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-154-1fe5209744ef>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.transform(p1, p2, coord[:, 0],

array([[126.85446233,  37.53239385],
       [126.84582053,  37.52953441],
       [126.83515161,  37.54452919],
       [126.84677474,  37.550881  ],
       [126.86544181,  37.54963623],
       [126.8338434 ,  37.53739023],
       [126.8379058 ,  37.54123382],
       [126.84464153,  37.54061593],
       [126.84337039,  37.53253912],
       [126.83262251,  37.53853002],
       [126.82344093,  37.5629473 ],
       [126.84601284,  37.56922499],
       [126.85170976,  37.56355296],
       [126.84900794,  37.56503063],
       [126.84654971,  37.53004647],
       [126.83436745,  37.55898087],
       [126.83352377,  37.54922434],
       [126.84483761,  37.53095495],
       [126.8187278 ,  37.56305994],
       [126.80761584,  37.56147927],
       [126.84046649,  37.57217309],
       [126.83931078,  37.56891629],
       [126.80286842,  37.57231513],
       [126.84607433,  37.56809849],
       [126.80977847,  37.57725935],
       [126.85114075,  37.56634733],
       [126.86901625,  37.55162883],
 

In [156]:
local_gangcho_3['경도']=result[:,0]
local_gangcho_3['위도']=result[:,1]

In [157]:
local_gangcho_4 = local_gangcho_3[['대피시설명', '주소', '평시활용 유형', '대피시설 면적', '경도', '위도']]

In [158]:
local_gangcho_4

,대피시설명,주소,평시활용 유형,대피시설 면적,경도,위도
0,성석교회 지하주차장,서울특별시 강서구 화곡동 836번지 13호,NaN,3889.00,126.854462,37.532394
1,KT 화곡지사 지하주차장,서울특별시 강서구 화곡동 893번지 1호,NaN,1126.00,126.845821,37.529534
2,서울큰나무병원,서울특별시 강서구 화곡동 1026번지 12호,NaN,1831.00,126.835152,37.544529
3,강서구청 지하상황실,서울특별시 강서구 화곡동 980번지 16호,NaN,770.25,126.846775,37.550881
4,강서보건소,서울특별시 강서구 염창동 275번지 12호,NaN,765.00,126.865442,37.549636
...,...,...,...,...,...,...
319,서초더샵포레아파트,서울특별시 서초구 내곡동 77번지,NaN,64966.00,127.061575,37.451064
320,서초포레스타 2단지아파트,서울특별시 서초구 내곡동 143번지,NaN,13352.00,127.060348,37.455025
321,서초포레스타 5단지아파트,서울특별시 서초구 신원동 374번지,NaN,28716.00,127.053517,37.452407
322,서초포레스타 6단지아파트,서울특별시 서초구 신원동 285번지,NaN,23172.00,127.052362,37.450309


# @여기까지가 서초 강서 파일 끝@

In [159]:
local_point

,시설번호,대피시설명,지번주소,도로명주소,대피시설 구분,평시활용 유형,대피시설 면적,대피시설 인원수,개방여부,데이터기준일자,제공기관명,제공기관코드,경도,위도,X좌표,Y좌표
0,367,한국전력강서양천지사,서울특별시 양천구 신정동 319-1,서울특별시 양천구 목동서로 313 (신정동 한국전력강서지점),공공용,기타,950.0,1151,Y,2021-05-20,서울특별시 양천구,3140000,126.868758,37.520086,188397.9745,546741.5100
1,368,출입국관리사무소,서울특별시 양천구 신정동 319-2,서울특별시 양천구 목동동로 151 (신정동 서울출입국관리사무소),공공용,기타,984.0,1192,Y,2021-05-20,서울특별시 양천구,3140000,126.869624,37.519247,188474.4014,546648.2851
2,369,평광교회,서울특별시 양천구 목동 909-2,서울특별시 양천구 목동서로 13 (목동),공공용,주차장,1272.0,1541,Y,2021-05-20,서울특별시 양천구,3140000,126.884365,37.538217,189780.1002,548752.0167
3,370,화성프라자,서울특별시 양천구 목동 909-5,서울특별시 양천구 목동동로 435 (목동),공공용,주차장,489.0,592,Y,2021-05-20,서울특별시 양천구,3140000,126.884848,37.537386,189822.7103,548659.7334
4,371,신월프라자,서울특별시 양천구 신월동 128,서울특별시 양천구 곰달래로5길 17 (신월동 신월프라자),공공용,주차장,7200.0,8727,Y,2021-05-20,서울특별시 양천구,3140000,126.833603,37.531110,185292.3726,547970.0039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908,2749,대진고등학교,서울특별시 노원구 하계동 55,서울특별시 노원구 공릉로62가길 47,공공용,NaN,1300.0,1575,N,2020-07-10,하계1동,3100000,127.076091,37.637805,206715.9964,559801.6042
2909,2750,영신여자고등학교 독서실,서울특별시 노원구 중계동 118,서울특별시 노원구 중계로16나길 13,공공용,NaN,264.0,320,N,2020-07-10,중계본동,3100000,127.083277,37.652454,207348.8001,561428.0047
2910,2751,상계5동주민센터,서울특별시 노원구 상계동 167-2,서울특별시 노원구 한글비석로 434,공공용,NaN,250.0,303,N,2020-07-10,상계5동,3100000,127.069366,37.662979,206120.3988,562595.2047
2911,2752,한신1차아파트,서울특별시 노원구 상계동 135,서울특별시 노원구 한글비석로46가길 16,공공용,NaN,12000.0,14545,N,2020-07-10,상계5동,3100000,127.068943,37.665844,206082.7983,562913.2039


## 0. 도로명주소가 기본, 없으면 지번주소 넣어서 새로운 컬럼 '주소' 생성

In [160]:
local_point['도로명주소'].isnull().sum()

21

In [161]:
local_point['주소'] = np.where(pd.notnull(local_point['도로명주소']) == True, local_point['도로명주소'], local_point['지번주소'])

In [162]:
local_point['주소'].isnull().sum()

0

# 1. 필요한 컬럼만 남기기

In [163]:
local_point_00 = local_point[['대피시설명', '주소', '평시활용 유형', '대피시설 면적', '경도', '위도']]

## 1-1 서초,강서 데이터와 합침

In [164]:
local_point_0_1 = pd.concat([local_point_00,local_gangcho_4])

In [165]:
local_point_1 = local_point_0_1.reset_index(drop=True)
local_point_1

,대피시설명,주소,평시활용 유형,대피시설 면적,경도,위도
0,한국전력강서양천지사,서울특별시 양천구 목동서로 313 (신정동 한국전력강서지점),기타,950.0,126.868758,37.520086
1,출입국관리사무소,서울특별시 양천구 목동동로 151 (신정동 서울출입국관리사무소),기타,984.0,126.869624,37.519247
2,평광교회,서울특별시 양천구 목동서로 13 (목동),주차장,1272.0,126.884365,37.538217
3,화성프라자,서울특별시 양천구 목동동로 435 (목동),주차장,489.0,126.884848,37.537386
4,신월프라자,서울특별시 양천구 곰달래로5길 17 (신월동 신월프라자),주차장,7200.0,126.833603,37.531110
...,...,...,...,...,...,...
3227,서초더샵포레아파트,서울특별시 서초구 내곡동 77번지,NaN,64966.0,127.061575,37.451064
3228,서초포레스타 2단지아파트,서울특별시 서초구 내곡동 143번지,NaN,13352.0,127.060348,37.455025
3229,서초포레스타 5단지아파트,서울특별시 서초구 신원동 374번지,NaN,28716.0,127.053517,37.452407
3230,서초포레스타 6단지아파트,서울특별시 서초구 신원동 285번지,NaN,23172.0,127.052362,37.450309


In [167]:
pd.set_option('display.max_rows', None)
seris = local_point_1['주소'].str.split(" ").str[1].value_counts()
frame1 = seris.to_frame()
frame1

,주소
관악구,201
노원구,197
강남구,194
강서구,166
중랑구,166
송파구,159
강동구,157
구로구,154
서초구,147
마포구,136


In [168]:
pd.options.display.max_rows = 10

# 2. 결측값 버리기 + 채우기

In [169]:
local_point_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3232 entries, 0 to 3231
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   대피시설명    3232 non-null   object 
 1   주소       3232 non-null   object 
 2   평시활용 유형  2160 non-null   object 
 3   대피시설 면적  3232 non-null   float64
 4   경도       3231 non-null   float64
 5   위도       3229 non-null   float64
dtypes: float64(3), object(3)
memory usage: 151.6+ KB


### (1) 경도, 위도 결측값 각 1,3개 발견! 그냥 버림.

In [170]:
del_list = set()

In [171]:
for i in local_point_1['위도'][local_point_1['위도'].isnull()].index :
    del_list.add(i)

In [172]:
for i in local_point_1['경도'][local_point_1['경도'].isnull()].index :
    del_list.add(i)

In [173]:
del_list

{269, 1075, 1986}

### (2) 평시활용 유형 결측값 채우고 버리기

In [174]:
local_point_1['평시활용 유형'].isnull().sum()

1072

In [175]:
local_point_1.loc[local_point_1["대피시설명"].str.contains('지하철|호선|역')
                        ,"평시활용 유형"] = "지하철"

In [176]:
local_point_1.loc[local_point_1["대피시설명"].str.contains('@|주차장|아파트|오피스텔|단지|빌|타워|타운|파크|시티|상가|프라자|호텔|백화점|교회')
                        ,"평시활용 유형"] = "주차장"

In [177]:
local_point_1.loc[local_point_1["대피시설명"].str.contains('센터|구청|회관|우체국|경찰서|병원|법원|복지관|도서관')
                        , "평시활용 유형"] = "공공시설"

In [178]:
local_point_1['평시활용 유형'].isnull().sum()

144

In [180]:
for i in local_point_1['평시활용 유형'][local_point_1['평시활용 유형'].isnull()].index :
    del_list.add(i)

In [184]:
local_point_2 = local_point_1.drop(del_list,axis=0)

In [185]:
local_point_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3085 entries, 0 to 3231
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   대피시설명    3085 non-null   object 
 1   주소       3085 non-null   object 
 2   평시활용 유형  3085 non-null   object 
 3   대피시설 면적  3085 non-null   float64
 4   경도       3085 non-null   float64
 5   위도       3085 non-null   float64
dtypes: float64(3), object(3)
memory usage: 168.7+ KB


In [186]:
local_point_2 = local_point_2.reset_index(drop=True)
local_point_2

,대피시설명,주소,평시활용 유형,대피시설 면적,경도,위도
0,한국전력강서양천지사,서울특별시 양천구 목동서로 313 (신정동 한국전력강서지점),기타,950.0,126.868758,37.520086
1,출입국관리사무소,서울특별시 양천구 목동동로 151 (신정동 서울출입국관리사무소),기타,984.0,126.869624,37.519247
2,평광교회,서울특별시 양천구 목동서로 13 (목동),주차장,1272.0,126.884365,37.538217
3,화성프라자,서울특별시 양천구 목동동로 435 (목동),주차장,489.0,126.884848,37.537386
4,신월프라자,서울특별시 양천구 곰달래로5길 17 (신월동 신월프라자),주차장,7200.0,126.833603,37.531110
...,...,...,...,...,...,...
3080,서초더샵포레아파트,서울특별시 서초구 내곡동 77번지,주차장,64966.0,127.061575,37.451064
3081,서초포레스타 2단지아파트,서울특별시 서초구 내곡동 143번지,주차장,13352.0,127.060348,37.455025
3082,서초포레스타 5단지아파트,서울특별시 서초구 신원동 374번지,주차장,28716.0,127.053517,37.452407
3083,서초포레스타 6단지아파트,서울특별시 서초구 신원동 285번지,주차장,23172.0,127.052362,37.450309


# 3. 필터

### (2) 경도, 위도 같은거 제거

In [187]:
local_point_3_2 = local_point_2.drop_duplicates(subset=['경도','위도'], keep= 'first')
local_point_3_2

,대피시설명,주소,평시활용 유형,대피시설 면적,경도,위도
0,한국전력강서양천지사,서울특별시 양천구 목동서로 313 (신정동 한국전력강서지점),기타,950.0,126.868758,37.520086
1,출입국관리사무소,서울특별시 양천구 목동동로 151 (신정동 서울출입국관리사무소),기타,984.0,126.869624,37.519247
2,평광교회,서울특별시 양천구 목동서로 13 (목동),주차장,1272.0,126.884365,37.538217
3,화성프라자,서울특별시 양천구 목동동로 435 (목동),주차장,489.0,126.884848,37.537386
4,신월프라자,서울특별시 양천구 곰달래로5길 17 (신월동 신월프라자),주차장,7200.0,126.833603,37.531110
...,...,...,...,...,...,...
3080,서초더샵포레아파트,서울특별시 서초구 내곡동 77번지,주차장,64966.0,127.061575,37.451064
3081,서초포레스타 2단지아파트,서울특별시 서초구 내곡동 143번지,주차장,13352.0,127.060348,37.455025
3082,서초포레스타 5단지아파트,서울특별시 서초구 신원동 374번지,주차장,28716.0,127.053517,37.452407
3083,서초포레스타 6단지아파트,서울특별시 서초구 신원동 285번지,주차장,23172.0,127.052362,37.450309


### (3) 면적 기준값 적용 제거 - 일단 100평(330m^2) 이상으로 필터링 - 회의 필요!!!!!!

In [188]:
local_point_3_3 = local_point_3_2[local_point_3_2['대피시설 면적'] >= 330]
local_point_3_3

,대피시설명,주소,평시활용 유형,대피시설 면적,경도,위도
0,한국전력강서양천지사,서울특별시 양천구 목동서로 313 (신정동 한국전력강서지점),기타,950.0,126.868758,37.520086
1,출입국관리사무소,서울특별시 양천구 목동동로 151 (신정동 서울출입국관리사무소),기타,984.0,126.869624,37.519247
2,평광교회,서울특별시 양천구 목동서로 13 (목동),주차장,1272.0,126.884365,37.538217
3,화성프라자,서울특별시 양천구 목동동로 435 (목동),주차장,489.0,126.884848,37.537386
4,신월프라자,서울특별시 양천구 곰달래로5길 17 (신월동 신월프라자),주차장,7200.0,126.833603,37.531110
...,...,...,...,...,...,...
3080,서초더샵포레아파트,서울특별시 서초구 내곡동 77번지,주차장,64966.0,127.061575,37.451064
3081,서초포레스타 2단지아파트,서울특별시 서초구 내곡동 143번지,주차장,13352.0,127.060348,37.455025
3082,서초포레스타 5단지아파트,서울특별시 서초구 신원동 374번지,주차장,28716.0,127.053517,37.452407
3083,서초포레스타 6단지아파트,서울특별시 서초구 신원동 285번지,주차장,23172.0,127.052362,37.450309


### (4) 경도, 위도 서울밖이상치 제거

In [189]:
local_point_3_4 = local_point_3_3[(local_point_3_3['경도']>=126.72) & (local_point_3_3['경도']<=127.27)
        & (local_point_3_3['위도']>=37.41) & (local_point_3_3['위도']<=37.72)]
local_point_3_4

,대피시설명,주소,평시활용 유형,대피시설 면적,경도,위도
0,한국전력강서양천지사,서울특별시 양천구 목동서로 313 (신정동 한국전력강서지점),기타,950.0,126.868758,37.520086
1,출입국관리사무소,서울특별시 양천구 목동동로 151 (신정동 서울출입국관리사무소),기타,984.0,126.869624,37.519247
2,평광교회,서울특별시 양천구 목동서로 13 (목동),주차장,1272.0,126.884365,37.538217
3,화성프라자,서울특별시 양천구 목동동로 435 (목동),주차장,489.0,126.884848,37.537386
4,신월프라자,서울특별시 양천구 곰달래로5길 17 (신월동 신월프라자),주차장,7200.0,126.833603,37.531110
...,...,...,...,...,...,...
3080,서초더샵포레아파트,서울특별시 서초구 내곡동 77번지,주차장,64966.0,127.061575,37.451064
3081,서초포레스타 2단지아파트,서울특별시 서초구 내곡동 143번지,주차장,13352.0,127.060348,37.455025
3082,서초포레스타 5단지아파트,서울특별시 서초구 신원동 374번지,주차장,28716.0,127.053517,37.452407
3083,서초포레스타 6단지아파트,서울특별시 서초구 신원동 285번지,주차장,23172.0,127.052362,37.450309


### (5) 평시활용 유형 필요한 것만 필터링

In [190]:
pd.set_option('display.max_rows', None)
local_point_3_4['평시활용 유형'].value_counts()

주차장       1987
공공시설       325
지하철        263
기타          35
지하주차장       35
시설주         16
체육관          5
교육장          4
회의교육         3
무용실          2
식당           2
학습실          2
강의실          2
지하보도         1
박물관          1
회의장          1
구내식당         1
창고 주차장       1
대피소식당        1
소강당          1
직원식당         1
고층건물         1
지하           1
교육장 창고       1
상업시설         1
교실           1
주민센터         1
보도           1
학습장          1
전시장          1
기타시설         1
지하상가         1
기계실          1
Name: 평시활용 유형, dtype: int64

In [191]:
pd.options.display.max_rows = 10

In [192]:
condition_list = ['주차장', '공공시설', '지하주차장']
local_point_3_5 = local_point_3_4[local_point_3_4['평시활용 유형'].isin(condition_list)]
local_point_3_5

,대피시설명,주소,평시활용 유형,대피시설 면적,경도,위도
2,평광교회,서울특별시 양천구 목동서로 13 (목동),주차장,1272.0,126.884365,37.538217
3,화성프라자,서울특별시 양천구 목동동로 435 (목동),주차장,489.0,126.884848,37.537386
4,신월프라자,서울특별시 양천구 곰달래로5길 17 (신월동 신월프라자),주차장,7200.0,126.833603,37.531110
6,보성아파트,서울특별시 양천구 오목로13길 33 (신월동 신월보성아파트),주차장,1207.0,126.844525,37.525881
7,우당아파트,서울특별시 양천구 오목로 90 (신월동),주차장,3894.0,126.847848,37.523044
...,...,...,...,...,...,...
3080,서초더샵포레아파트,서울특별시 서초구 내곡동 77번지,주차장,64966.0,127.061575,37.451064
3081,서초포레스타 2단지아파트,서울특별시 서초구 내곡동 143번지,주차장,13352.0,127.060348,37.455025
3082,서초포레스타 5단지아파트,서울특별시 서초구 신원동 374번지,주차장,28716.0,127.053517,37.452407
3083,서초포레스타 6단지아파트,서울특별시 서초구 신원동 285번지,주차장,23172.0,127.052362,37.450309


# 4. 지도에 찍기

In [199]:
pd.set_option('display.max_rows', None)
seris = local_point_3_5['주소'].str.split(" ").str[1].value_counts()
frame2 = seris.to_frame()

In [201]:
addframe = pd.concat([frame1,frame2],axis=1)
addframe

,주소,주소
관악구,201,128
노원구,197,111
강남구,194,139
강서구,166,118
중랑구,166,144
송파구,159,118
강동구,157,140
구로구,154,126
서초구,147,106
마포구,136,88


In [202]:
pd.options.display.max_rows = 10

In [203]:
import folium
m = folium.Map(location=[37.5642135, 127.0016985],zoom_start = 12)
m

In [204]:
from folium.plugins import MarkerCluster

coords = local_point_3_5[['위도', '경도']]

marker_cluster = MarkerCluster().add_to(m)
for lat, long in zip(coords['위도'], coords['경도']):
    folium.Marker([lat, long],icon = folium.Icon(color="green")).add_to(marker_cluster)
m

In [205]:
local_point_3_5.to_csv('final_file_0417.csv',encoding='cp949')